In [14]:
import pandas as pd
import numpy as np
other_out = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/regimen_cancer_level_dataset.csv')
other_out['drugs_combined'] = other_out[['drugs_drug_1', 'drugs_drug_2', 'drugs_drug_3', 'drugs_drug_4', 'drugs_drug_5']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
other_out['drugs_combined'].value_counts()
data_ib = pd.read_csv('../data/crc_raw/cBioPortal_files/data_timeline_treatment.txt', sep='\t')
reg_df = data_ib['REGIMEN'].value_counts()
reg_df.head(20)

REGIMEN
Fluorouracil, Leucovorin Calcium, Oxaliplatin                                           3054
Bevacizumab, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                 2488
Bevacizumab, Fluorouracil, Leucovorin Calcium, Oxaliplatin                              1880
Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                              1227
Capecitabine                                                                             322
Cetuximab, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                    316
Bevacizumab, Fluorouracil, Leucovorin Calcium                                            282
Fluorouracil, Leucovorin Calcium                                                         260
Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium, Panitumumab                  224
Floxuridine                                                                              220
Bevacizumab, Capecitabine                                     

In [15]:
data_ib = data_ib.loc[(data_ib['REGIMEN'].str.contains('cetux', case=False, na=False)) | (data_ib['REGIMEN'].str.contains('panitu', case=False, na=False))]
data_ib['REGIMEN'].value_counts()


REGIMEN
Cetuximab, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                 316
Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium, Panitumumab               224
Irinotecan Hydrochloride, Panitumumab                                                 148
Cetuximab, Irinotecan Hydrochloride                                                   142
Fluorouracil, Leucovorin Calcium, Oxaliplatin, Panitumumab                             80
Cetuximab, Fluorouracil, Leucovorin Calcium, Oxaliplatin                               72
Bevacizumab, Cetuximab, Irinotecan Hydrochloride                                       36
Panitumumab                                                                            36
Dabrafenib, Panitumumab, Trametinib                                                    24
Investigational Drug, Irinotecan Hydrochloride, Panitumumab                            21
Cetuximab, Fluorouracil, Leucovorin Calcium                                            18
Ce

In [16]:
other_out.head()

,cohort,record_id,institution,ca_seq,regimen_number,regimen_number_within_cancer,redcap_ca_index,drugs_num,drugs_inst,drugs_firstinst,...,ttnt_any_ca_status,ttnt_any_ca_days,ttnt_any_ca_mos,ttnt_any_ca_yrs,ttnt_ca_seq_status,ttnt_ca_seq_days,ttnt_ca_seq_mos,ttnt_ca_seq_yrs,release_version,drugs_combined
0,CRC,GENIE-DFCI-000048,DFCI,0,1,1,Yes,2,At the internal/native institution only,NaN,...,0,273,8.980263,0.747433,0,273,8.980263,0.747433,2.0-public,"Fluorouracil(5FU,5Fluorouracil,5Fluracil,AccuS..."
1,CRC,GENIE-DFCI-000147,DFCI,0,1,1,Yes,1,At external institution only,NaN,...,1,181,5.953947,0.495551,1,181,5.953947,0.495551,2.0-public,"Capecitabine(Ro091978 000,Xeloda)"
2,CRC,GENIE-DFCI-000147,DFCI,0,2,2,Yes,2,At external institution only,NaN,...,1,275,9.046053,0.752909,1,275,9.046053,0.752909,2.0-public,"Capecitabine(Ro091978 000,Xeloda),Oxaliplatin(..."
3,CRC,GENIE-DFCI-000147,DFCI,0,3,3,Yes,4,At external institution only,NaN,...,1,473,15.559211,1.295003,1,473,15.559211,1.295003,2.0-public,"Fluorouracil(5FU,5Fluorouracil,5Fluracil,AccuS..."
4,CRC,GENIE-DFCI-000147,DFCI,0,4,4,Yes,1,At the internal/native institution only,NaN,...,1,282,9.276316,0.772074,1,282,9.276316,0.772074,2.0-public,Trifluridine and Tipiracil Hydrochloride(Lonsu...


In [17]:
ib_regimens = data_ib['REGIMEN'].unique()
len(ib_regimens)
ib_list = [reg.split(',') for reg in ib_regimens]
ib_list
#if 'Investigational Drug' is one of the drugs, drop it from the list
for i in ib_list:
    if 'Investigational Drug' in i:
        i.remove('Investigational Drug')
    elif ' Investigational Drug' in i:
        i.remove(' Investigational Drug')
#strip leading and trailing spaces
ib_list = [[drug.strip() for drug in reg] for reg in ib_list]
ib_list
treatment_list = data_ib['REGIMEN'].unique()
#save to csv
# pd.DataFrame(treatment_list).to_csv('../data/egfr_list.csv', index=False)
#find the rows in other_out that contain each of the drugs in each element of ib_list
ib_out = pd.DataFrame()
for i in ib_list:
    if len(i) == 1:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False)]
        #make a new column with the list of drugs in each row
        new_df['drugs_list'] = str(i[0])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 2:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 3:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 4:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False) & other_out['drugs_combined'].str.contains(i[3], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2]) + ', ' + str(i[3])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
ib_out.shape
keep = ['record_id', 'institution', 'drugs_num', 'drugs_list', 'len_reg', 'dx_drug_start_int_1', 'os_g_status', 'tt_os_g_mos', 'pfs_m_g_status', 'tt_pfs_m_g_mos']
clin = ib_out.copy()
ib_out = ib_out[keep]
ib_out.sort_values(by=['record_id', 'drugs_num', 'len_reg'], inplace=True)
ib_out.head()


/tmp/ipykernel_2832573/3481826189.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2])
/tmp/ipykernel_2832573/3481826189.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['len_reg'] = len(i)
/tmp/ipykernel_2832573/3481826189.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

,record_id,institution,drugs_num,drugs_list,len_reg,dx_drug_start_int_1,os_g_status,tt_os_g_mos,pfs_m_g_status,tt_pfs_m_g_mos
34,GENIE-DFCI-000971,DFCI,3,Cetuximab,1,463,1,11.546053,1.0,5.723684
34,GENIE-DFCI-000971,DFCI,3,Cetuximab,1,463,1,11.546053,1.0,5.723684
34,GENIE-DFCI-000971,DFCI,3,"Cetuximab, Irinotecan Hydrochloride",2,463,1,11.546053,1.0,5.723684
34,GENIE-DFCI-000971,DFCI,3,"Bevacizumab, Cetuximab, Irinotecan Hydrochloride",3,463,1,11.546053,1.0,5.723684
58,GENIE-DFCI-001292,DFCI,4,Panitumumab,1,318,1,10.493421,1.0,4.144737


In [18]:
ib_out['tt_os_g_mos'].describe()

count    2232.000000
mean       20.280445
std        20.039872
min         0.098684
25%         6.735197
50%        14.506579
75%        27.319079
max       175.263158
Name: tt_os_g_mos, dtype: float64

In [19]:
ib_out['tt_pfs_m_g_mos'].describe()

count    2170.000000
mean        6.432786
std        10.583664
min         0.000000
25%         1.842105
50%         4.375000
75%         8.190789
max       145.460526
Name: tt_pfs_m_g_mos, dtype: float64

In [20]:
ib_out['os_g_status'].value_counts()

os_g_status
1    1375
0     857
Name: count, dtype: int64

In [21]:
#if os is tt_os is >12, set os_g_status to 0
ib_out.loc[ib_out['tt_os_g_mos'] > 12, 'os_g_status'] = 0
ib_out['os_g_status'].value_counts()

os_g_status
0    1230
1    1002
Name: count, dtype: int64

In [22]:
ib_out['pfs_m_g_status'].value_counts()

pfs_m_g_status
1.0    1187
0.0     983
Name: count, dtype: int64

In [23]:
#if pfs is tt_pfs is >6, set pfs_m_g_status to 0
ib_out.loc[ib_out['tt_pfs_m_g_mos'] > 6, 'pfs_m_g_status'] = 0
ib_out['pfs_m_g_status'].value_counts()

pfs_m_g_status
0.0    1460
1.0     710
Name: count, dtype: int64

In [24]:

print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num', 'drugs_list'], keep='first')
print(ib_out.shape)
print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num'], keep='last')
print(ib_out.shape)
ib_out['drugs_list'].value_counts()
ib_out.reset_index(inplace=True, drop=True)
ib_out['sample_id'] = np.NaN
ib_out['diff'] = np.NaN
panel = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/cancer_panel_test_level_dataset.csv', index_col='cpt_genie_sample_id')
keep = ['record_id', 'dx_cpt_rep_days']
panel = panel[keep]
panel.sort_values(by=['record_id'], inplace=True)

for i in range(len(ib_out)):
    
    record_id = ib_out.iloc[i]['record_id']
    panel_rows = panel.loc[panel['record_id'] == record_id]
    panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
    if len(panel_rows) == 1:
        ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
        ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
    elif len(panel_rows) > 1:
        #use the row with the smallest difference, as long as it's less than 0, unless they are all positive
        if len(panel_rows.loc[panel_rows['diff'] < 0]) == 0:
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
        else:
            panel_rows = panel_rows.loc[panel_rows['diff'] < 0]
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
ib_out.head()
ib_out.drop(['drugs_num', 'len_reg', 'dx_drug_start_int_1', 'diff'], axis=1, inplace=True)
ib_out.head()


(2232, 10)
(1269, 10)
(1269, 10)
(368, 10)


/tmp/ipykernel_2832573/475263278.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_2832573/475263278.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_2832573/475263278.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,record_id,institution,drugs_list,os_g_status,tt_os_g_mos,pfs_m_g_status,tt_pfs_m_g_mos,sample_id
0,GENIE-DFCI-000971,DFCI,"Bevacizumab, Cetuximab, Irinotecan Hydrochloride",1,11.546053,1.0,5.723684,GENIE-DFCI-000971-10958
1,GENIE-DFCI-001292,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin,...",1,10.493421,1.0,4.144737,GENIE-DFCI-001292-7406
2,GENIE-DFCI-001463,DFCI,Cetuximab,1,9.703947,0.0,5.756579,GENIE-DFCI-001463-9650
3,GENIE-DFCI-001463,DFCI,"Cetuximab, Irinotecan Hydrochloride",1,3.947368,1.0,2.927632,GENIE-DFCI-001463-9650
4,GENIE-DFCI-002507,DFCI,"Fluorouracil, Irinotecan Hydrochloride, Leucov...",0,35.427632,1.0,5.953947,GENIE-DFCI-002507-4815


In [25]:

ib_out['pfs_m_g_status'].value_counts()


pfs_m_g_status
0.0    234
1.0    128
Name: count, dtype: int64

In [26]:
ib_out['os_g_status'].value_counts()


os_g_status
1    188
0    180
Name: count, dtype: int64

In [27]:
ib_out.to_csv('../data/crc_egfr_out.csv', index=True)

In [31]:
other_out = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/regimen_cancer_level_dataset.csv')
other_out['drugs_combined'] = other_out[['drugs_drug_1', 'drugs_drug_2', 'drugs_drug_3', 'drugs_drug_4', 'drugs_drug_5']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
other_out['drugs_combined'].value_counts()
data_ib = pd.read_csv('../data/crc_raw/cBioPortal_files/data_timeline_treatment.txt', sep='\t')
#replace 'bevacizumab' in 'REGIMEN' with 'bev'
data_ib['REGIMEN'] = data_ib['REGIMEN'].str.replace('bevacizumab', 'bev')
data_ib['REGIMEN'] = data_ib['REGIMEN'].str.replace('Bevacizumab', 'bev')
data_ib = data_ib.loc[(data_ib['REGIMEN'].str.contains('fluorouracil', case=False, na=False))]
data_ib['REGIMEN'].value_counts()


REGIMEN
Fluorouracil, Leucovorin Calcium, Oxaliplatin                                                             3054
bev, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                                           2488
bev, Fluorouracil, Leucovorin Calcium, Oxaliplatin                                                        1880
Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                                                1227
Cetuximab, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                                      316
bev, Fluorouracil, Leucovorin Calcium                                                                      282
Fluorouracil, Leucovorin Calcium                                                                           260
Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium, Panitumumab                                    224
Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium, Oxaliplatin                                 

In [32]:
data_ib = data_ib.loc[((data_ib['REGIMEN'].str.contains('Fluorouracil', case=False, na=False) &
                        data_ib['REGIMEN'].str.contains('Oxaliplatin', case=False, na=False)) |
                        (data_ib['REGIMEN'].str.contains('Capecitabine', case=False, na=False) &
                        data_ib['REGIMEN'].str.contains('Oxaliplatin', case=False, na=False))) &
                       ~(data_ib['REGIMEN'].str.contains('mab', case=False, na=False) | 
                         data_ib['REGIMEN'].str.contains('ib', case=False, na=False) |
                         data_ib['REGIMEN'].str.contains('nib', case=False, na=False) |
                         data_ib['REGIMEN'].str.contains('investigational', case=False, na=False) |
                         data_ib['REGIMEN'].str.contains('everoli', case=False, na=False) |
                         data_ib['REGIMEN'].str.contains('iri', case=False, na=False))]
data_ib['REGIMEN'].value_counts()


REGIMEN
Fluorouracil, Leucovorin Calcium, Oxaliplatin                               3054
bev, Fluorouracil, Leucovorin Calcium, Oxaliplatin                          1880
bev, Fluorouracil, Oxaliplatin                                                18
Capecitabine, Fluorouracil, Leucovorin Calcium, Oxaliplatin                   16
Fluorouracil, Oxaliplatin                                                     14
bev, Capecitabine, Fluorouracil, Leucovorin Calcium, Oxaliplatin              10
Fluorouracil, Gemcitabine Hydrochloride, Leucovorin Calcium, Oxaliplatin       4
Name: count, dtype: int64

In [34]:
ib_regimens = data_ib['REGIMEN'].unique()
len(ib_regimens)
ib_list = [reg.split(',') for reg in ib_regimens]
ib_list
#if 'Investigational Drug' is one of the drugs, drop it from the list
for i in ib_list:
    if 'Investigational Drug' in i:
        i.remove('Investigational Drug')
    elif ' Investigational Drug' in i:
        i.remove(' Investigational Drug')
#strip leading and trailing spaces
ib_list = [[drug.strip() for drug in reg] for reg in ib_list]
ib_list
treatment_list = data_ib['REGIMEN'].unique()


#find the rows in other_out that contain each of the drugs in each element of ib_list
ib_out = pd.DataFrame()
for i in ib_list:
    if len(i) == 1:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False)]
        #make a new column with the list of drugs in each row
        new_df['drugs_list'] = str(i[0])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 2:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 3:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 4:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False) & other_out['drugs_combined'].str.contains(i[3], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2]) + ', ' + str(i[3])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
ib_out.shape
ib_out.head()
cols = ib_out.columns.to_list()
ib_out['pfs_i_and_m_g_status'].value_counts()
keep = ['record_id', 'institution', 'drugs_num', 'drugs_list', 'len_reg', 'dx_drug_start_int_1', 'os_g_status', 'tt_os_g_mos', 'pfs_m_g_status', 'tt_pfs_m_g_mos']
clin = ib_out.copy()
ib_out = ib_out[keep]
ib_out.sort_values(by=['record_id', 'drugs_num', 'len_reg'], inplace=True)
ib_out.head()

print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num', 'drugs_list'], keep='first')
print(ib_out.shape)
print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num'], keep='last')
print(ib_out.shape)
ib_out['drugs_list'].value_counts()
ib_out.reset_index(inplace=True, drop=True)
ib_out['sample_id'] = np.NaN
ib_out['diff'] = np.NaN
panel = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/cancer_panel_test_level_dataset.csv', index_col='cpt_genie_sample_id')
keep = ['record_id', 'dx_cpt_rep_days']
panel = panel[keep]
panel.sort_values(by=['record_id'], inplace=True)

for i in range(len(ib_out)):
    
    record_id = ib_out.iloc[i]['record_id']
    panel_rows = panel.loc[panel['record_id'] == record_id]
    panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
    if len(panel_rows) == 1:
        ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
        ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
    elif len(panel_rows) > 1:
        #use the row with the smallest difference, as long as it's less than 0, unless they are all positive
        if len(panel_rows.loc[panel_rows['diff'] < 0]) == 0:
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
        else:
            panel_rows = panel_rows.loc[panel_rows['diff'] < 0]
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
ib_out.head()
ib_out.drop(['drugs_num', 'len_reg', 'dx_drug_start_int_1', 'diff'], axis=1, inplace=True)
ib_out.head()


/tmp/ipykernel_2832573/2357693051.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2])
/tmp/ipykernel_2832573/2357693051.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['len_reg'] = len(i)
/tmp/ipykernel_2832573/2357693051.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

(4309, 10)
(3509, 10)
(3509, 10)
(1346, 10)


/tmp/ipykernel_2832573/2357693051.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_2832573/2357693051.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_2832573/2357693051.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,record_id,institution,drugs_list,os_g_status,tt_os_g_mos,pfs_m_g_status,tt_pfs_m_g_mos,sample_id
0,GENIE-DFCI-000233,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",1,100.986842,1.0,31.578947,GENIE-DFCI-000233-7633
1,GENIE-DFCI-000247,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",1,35.789474,0.0,3.914474,GENIE-DFCI-000247-11259
2,GENIE-DFCI-000306,DFCI,"bev, Fluorouracil, Leucovorin Calcium, Oxalipl...",1,38.125000,1.0,12.960526,GENIE-DFCI-000306-5914
3,GENIE-DFCI-000738,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",0,51.578947,NaN,NaN,GENIE-DFCI-000738-11390
4,GENIE-DFCI-000924,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",0,65.361842,NaN,NaN,GENIE-DFCI-000924-6164


In [36]:
ib_out['tt_pfs_m_g_mos'].describe()

count    1015.000000
mean       10.074281
std        12.992356
min         0.000000
25%         3.207237
50%         5.723684
75%        10.279605
max       109.901316
Name: tt_pfs_m_g_mos, dtype: float64

In [37]:
ib_out['tt_os_g_mos'].describe()

count    1346.000000
mean       37.832564
std        28.130697
min         0.065789
25%        16.151316
50%        33.223684
75%        51.784539
max       187.631579
Name: tt_os_g_mos, dtype: float64

In [40]:
print(ib_out['pfs_m_g_status'].value_counts())
ib_out.loc[ib_out['tt_pfs_m_g_mos'] > 12, 'pfs_m_g_status'] = 0
print(ib_out['pfs_m_g_status'].value_counts())

pfs_m_g_status
0.0    653
1.0    362
Name: count, dtype: int64
pfs_m_g_status
0.0    746
1.0    269
Name: count, dtype: int64


In [42]:
print(ib_out['os_g_status'].value_counts())
ib_out.loc[ib_out['tt_os_g_mos'] > 36, 'os_g_status'] = 0
print(ib_out['os_g_status'].value_counts())

os_g_status
1    715
0    631
Name: count, dtype: int64
os_g_status
0    857
1    489
Name: count, dtype: int64


In [43]:
ib_out.to_csv('../data/crc_folfox_out.csv', index=True)

In [44]:
other_out = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/regimen_cancer_level_dataset.csv')
other_out['drugs_combined'] = other_out[['drugs_drug_1', 'drugs_drug_2', 'drugs_drug_3', 'drugs_drug_4', 'drugs_drug_5']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
other_out['drugs_combined'].value_counts()
data_ib = pd.read_csv('../data/crc_raw/cBioPortal_files/data_timeline_treatment.txt', sep='\t')
data_ib = data_ib.loc[(data_ib['REGIMEN'].str.contains('trifluridine', case=False, na=False))]
data_ib['REGIMEN'].value_counts()


REGIMEN
Trifluridine and Tipiracil Hydrochloride                                           154
Bevacizumab, Trifluridine and Tipiracil Hydrochloride                                6
Cetuximab, Trifluridine and Tipiracil Hydrochloride, Vemurafenib                     3
Irinotecan Hydrochloride, Ramucirumab, Trifluridine and Tipiracil Hydrochloride      3
Irinotecan Hydrochloride, Trifluridine and Tipiracil Hydrochloride                   2
Name: count, dtype: int64

In [50]:
ib_regimens = data_ib['REGIMEN'].unique()
len(ib_regimens)
ib_list = [reg.split(',') for reg in ib_regimens]
ib_list
#if 'Investigational Drug' is one of the drugs, drop it from the list
for i in ib_list:
    if 'Investigational Drug' in i:
        i.remove('Investigational Drug')
    elif ' Investigational Drug' in i:
        i.remove(' Investigational Drug')
#strip leading and trailing spaces
ib_list = [[drug.strip() for drug in reg] for reg in ib_list]
ib_list
treatment_list = data_ib['REGIMEN'].unique()


#find the rows in other_out that contain each of the drugs in each element of ib_list
ib_out = pd.DataFrame()
for i in ib_list:
    if len(i) == 1:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False)]
        #make a new column with the list of drugs in each row
        new_df['drugs_list'] = str(i[0])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 2:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 3:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 4:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False) & other_out['drugs_combined'].str.contains(i[3], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2]) + ', ' + str(i[3])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
ib_out.shape
ib_out.head()
cols = ib_out.columns.to_list()
ib_out['pfs_i_and_m_g_status'].value_counts()
keep = ['record_id', 'institution', 'drugs_num', 'drugs_list', 'len_reg', 'dx_drug_start_int_1', 'os_g_status', 'tt_os_g_mos', 'pfs_m_g_status', 'tt_pfs_m_g_mos']
clin = ib_out.copy()
ib_out = ib_out[keep]
ib_out.sort_values(by=['record_id', 'drugs_num', 'len_reg'], inplace=True)
ib_out.head()

print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num', 'drugs_list'], keep='first')
print(ib_out.shape)
print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num'], keep='last')
print(ib_out.shape)
ib_out['drugs_list'].value_counts()
ib_out.reset_index(inplace=True, drop=True)
ib_out['sample_id'] = np.NaN
ib_out['diff'] = np.NaN
panel = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/cancer_panel_test_level_dataset.csv', index_col='cpt_genie_sample_id')
keep = ['record_id', 'dx_cpt_rep_days']
panel = panel[keep]
panel.sort_values(by=['record_id'], inplace=True)

for i in range(len(ib_out)):
    
    record_id = ib_out.iloc[i]['record_id']
    panel_rows = panel.loc[panel['record_id'] == record_id]
    panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
    if len(panel_rows) == 1:
        ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
        ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
    elif len(panel_rows) > 1:
        #use the row with the smallest difference, as long as it's less than 0, unless they are all positive
        if len(panel_rows.loc[panel_rows['diff'] < 0]) == 0:
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
        else:
            panel_rows = panel_rows.loc[panel_rows['diff'] < 0]
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
ib_out.head()
ib_out.drop(['drugs_num', 'len_reg', 'dx_drug_start_int_1', 'diff'], axis=1, inplace=True)
ib_out.head()


/tmp/ipykernel_2832573/2357693051.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['drugs_list'] = str(i[0])
/tmp/ipykernel_2832573/2357693051.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['len_reg'] = len(i)
/tmp/ipykernel_2832573/2357693051.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

(167, 10)
(162, 10)
(162, 10)
(155, 10)


/tmp/ipykernel_2832573/2357693051.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_2832573/2357693051.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_2832573/2357693051.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,record_id,institution,drugs_list,os_g_status,tt_os_g_mos,pfs_m_g_status,tt_pfs_m_g_mos,sample_id
0,GENIE-DFCI-000147,DFCI,Trifluridine and Tipiracil Hydrochloride,1,9.276316,1.0,4.407895,GENIE-DFCI-000147-6898
1,GENIE-DFCI-000971,DFCI,Trifluridine and Tipiracil Hydrochloride,1,2.368421,1.0,0.197368,GENIE-DFCI-000971-10958
2,GENIE-DFCI-001199,DFCI,Trifluridine and Tipiracil Hydrochloride,1,5.526316,1.0,2.532895,GENIE-DFCI-001199-5906
3,GENIE-DFCI-001292,DFCI,Trifluridine and Tipiracil Hydrochloride,1,3.125000,1.0,1.381579,GENIE-DFCI-001292-7406
4,GENIE-DFCI-002499,DFCI,Trifluridine and Tipiracil Hydrochloride,1,7.960526,1.0,0.460526,GENIE-DFCI-002499-10699


In [51]:
ib_out['tt_pfs_m_g_mos'].describe()


count    153.000000
mean       2.667914
std        2.248885
min        0.000000
25%        1.282895
50%        2.072368
75%        3.157895
max       15.526316
Name: tt_pfs_m_g_mos, dtype: float64

In [52]:
ib_out['tt_os_g_mos'].describe()


count    155.000000
mean       7.502971
std        6.959865
min        0.000000
25%        2.697368
50%        5.328947
75%        9.391447
max       43.223684
Name: tt_os_g_mos, dtype: float64

In [53]:
print(ib_out['pfs_m_g_status'].value_counts())
ib_out.loc[ib_out['tt_pfs_m_g_mos'] > 6, 'pfs_m_g_status'] = 0
print(ib_out['pfs_m_g_status'].value_counts())


pfs_m_g_status
1.0    124
0.0     29
Name: count, dtype: int64
pfs_m_g_status
1.0    114
0.0     39
Name: count, dtype: int64


In [54]:
print(ib_out['os_g_status'].value_counts())
ib_out.loc[ib_out['tt_os_g_mos'] > 12, 'os_g_status'] = 0
print(ib_out['os_g_status'].value_counts())


os_g_status
1    117
0     38
Name: count, dtype: int64
os_g_status
1    96
0    59
Name: count, dtype: int64


In [55]:
ib_out.to_csv('../data/crc_tri_out.csv', index=True)

In [63]:
other_out = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/regimen_cancer_level_dataset.csv')
other_out['drugs_combined'] = other_out[['drugs_drug_1', 'drugs_drug_2', 'drugs_drug_3', 'drugs_drug_4', 'drugs_drug_5']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
other_out['drugs_combined'].value_counts()
data_ib = pd.read_csv('../data/crc_raw/cBioPortal_files/data_timeline_treatment.txt', sep='\t')
#replace 'bevacizumab' in 'REGIMEN' with 'bev'
data_ib['REGIMEN'] = data_ib['REGIMEN'].str.replace('bevacizumab', 'bev')
data_ib['REGIMEN'] = data_ib['REGIMEN'].str.replace('Bevacizumab', 'bev')

data_ib = data_ib.loc[((data_ib['REGIMEN'].str.contains('Fluorouracil', case=False, na=False) &
                        data_ib['REGIMEN'].str.contains('Irinotecan', case=False, na=False)) |
                        (data_ib['REGIMEN'].str.contains('Capecitabine', case=False, na=False) &
                        data_ib['REGIMEN'].str.contains('Irinotecan', case=False, na=False))) &
                       ~(data_ib['REGIMEN'].str.contains('mab', case=False, na=False) | 
                         data_ib['REGIMEN'].str.contains('ib', case=False, na=False) |
                         data_ib['REGIMEN'].str.contains('nib', case=False, na=False) |
                         data_ib['REGIMEN'].str.contains('investigational', case=False, na=False) |
                         data_ib['REGIMEN'].str.contains('everoli', case=False, na=False) |
                         data_ib['REGIMEN'].str.contains('oxali', case=False, na=False))]
data_ib['REGIMEN'].value_counts()

REGIMEN
bev, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                  2488
Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium                       1227
bev, Fluorouracil, Irinotecan Hydrochloride                                        45
bev, Capecitabine, Irinotecan Hydrochloride                                        45
Fluorouracil, Irinotecan Hydrochloride                                             14
Capecitabine, Irinotecan Hydrochloride                                             12
bev, Fluorouracil, Irinotecan liposome, Leucovorin Calcium                         12
Fluorouracil, Irinotecan liposome, Leucovorin Calcium                               6
bev, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium, Temozolomide       5
bev, Capecitabine, Fluorouracil, Irinotecan Hydrochloride, Leucovorin Calcium       5
bev, Cisplatin, Fluorouracil, Irinotecan Hydrochloride                              4
Name: count, dtype: int64

In [65]:
ib_regimens = data_ib['REGIMEN'].unique()
len(ib_regimens)
ib_list = [reg.split(',') for reg in ib_regimens]
ib_list
#if 'Investigational Drug' is one of the drugs, drop it from the list
for i in ib_list:
    if 'Investigational Drug' in i:
        i.remove('Investigational Drug')
    elif ' Investigational Drug' in i:
        i.remove(' Investigational Drug')
#strip leading and trailing spaces
ib_list = [[drug.strip() for drug in reg] for reg in ib_list]
ib_list
treatment_list = data_ib['REGIMEN'].unique()


#find the rows in other_out that contain each of the drugs in each element of ib_list
ib_out = pd.DataFrame()
for i in ib_list:
    if len(i) == 1:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False)]
        #make a new column with the list of drugs in each row
        new_df['drugs_list'] = str(i[0])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 2:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 3:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
    elif len(i) == 4:
        new_df = other_out.loc[other_out['drugs_combined'].str.contains(i[0], case=False, na=False) & other_out['drugs_combined'].str.contains(i[1], case=False, na=False) & other_out['drugs_combined'].str.contains(i[2], case=False, na=False) & other_out['drugs_combined'].str.contains(i[3], case=False, na=False)]
        new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2]) + ', ' + str(i[3])
        new_df['len_reg'] = len(i)
        ib_out = pd.concat([ib_out, new_df])
ib_out.shape
ib_out.head()
cols = ib_out.columns.to_list()
ib_out['pfs_i_and_m_g_status'].value_counts()
keep = ['record_id', 'institution', 'drugs_num', 'drugs_list', 'len_reg', 'dx_drug_start_int_1', 'os_g_status', 'tt_os_g_mos', 'pfs_m_g_status', 'tt_pfs_m_g_mos']
clin = ib_out.copy()
ib_out = ib_out[keep]
ib_out.sort_values(by=['record_id', 'drugs_num', 'len_reg'], inplace=True)
ib_out.head()

print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num', 'drugs_list'], keep='first')
print(ib_out.shape)
print(ib_out.shape)
ib_out = ib_out.drop_duplicates(subset=['record_id', 'drugs_num'], keep='last')
print(ib_out.shape)
ib_out['drugs_list'].value_counts()
ib_out.reset_index(inplace=True, drop=True)
ib_out['sample_id'] = np.NaN
ib_out['diff'] = np.NaN
panel = pd.read_csv('../data/crc_raw/CRC_2.0-public_clinical_data/cancer_panel_test_level_dataset.csv', index_col='cpt_genie_sample_id')
keep = ['record_id', 'dx_cpt_rep_days']
panel = panel[keep]
panel.sort_values(by=['record_id'], inplace=True)

for i in range(len(ib_out)):
    
    record_id = ib_out.iloc[i]['record_id']
    panel_rows = panel.loc[panel['record_id'] == record_id]
    panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
    if len(panel_rows) == 1:
        ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
        ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
    elif len(panel_rows) > 1:
        #use the row with the smallest difference, as long as it's less than 0, unless they are all positive
        if len(panel_rows.loc[panel_rows['diff'] < 0]) == 0:
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
        else:
            panel_rows = panel_rows.loc[panel_rows['diff'] < 0]
            panel_rows.sort_values(by=['diff'], inplace=True)
            ib_out.iloc[i, ib_out.columns.get_loc('sample_id')] = panel_rows.index.values[0]
            ib_out.iloc[i, ib_out.columns.get_loc('diff')] = panel_rows['diff'].values[0]
ib_out.head()
ib_out.drop(['drugs_num', 'len_reg', 'dx_drug_start_int_1', 'diff'], axis=1, inplace=True)
ib_out.head()


/tmp/ipykernel_2832573/2357693051.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['drugs_list'] = str(i[0]) + ', ' + str(i[1]) + ', ' + str(i[2]) + ', ' + str(i[3])
/tmp/ipykernel_2832573/2357693051.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['len_reg'] = len(i)
/tmp/ipykernel_2832573/2357693051.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

(3997, 10)
(2877, 10)
(2877, 10)
(946, 10)


/tmp/ipykernel_2832573/2357693051.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_2832573/2357693051.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_rows['diff'] = panel_rows['dx_cpt_rep_days'] - ib_out.iloc[i]['dx_drug_start_int_1']
/tmp/ipykernel_2832573/2357693051.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,record_id,institution,drugs_list,os_g_status,tt_os_g_mos,pfs_m_g_status,tt_pfs_m_g_mos,sample_id
0,GENIE-DFCI-000147,DFCI,"bev, Fluorouracil, Irinotecan Hydrochloride, L...",1,24.835526,1.0,6.348684,GENIE-DFCI-000147-6898
1,GENIE-DFCI-000233,DFCI,"bev, Fluorouracil, Irinotecan Hydrochloride, L...",1,46.611842,0.0,14.046053,GENIE-DFCI-000233-7633
2,GENIE-DFCI-000247,DFCI,"Fluorouracil, Irinotecan Hydrochloride, Leucov...",1,21.513158,1.0,21.513158,GENIE-DFCI-000247-11259
3,GENIE-DFCI-000306,DFCI,"bev, Fluorouracil, Irinotecan Hydrochloride, L...",1,20.098684,1.0,8.684211,GENIE-DFCI-000306-5914
4,GENIE-DFCI-000738,DFCI,"bev, Fluorouracil, Irinotecan Hydrochloride, L...",0,46.315789,0.0,6.710526,GENIE-DFCI-000738-11390


In [67]:
ib_out['tt_pfs_m_g_mos'].describe()


count    919.000000
mean       7.295115
std        9.064706
min        0.000000
25%        2.302632
50%        5.197368
75%        8.865132
max      134.638158
Name: tt_pfs_m_g_mos, dtype: float64

In [68]:
ib_out['tt_os_g_mos'].describe()


count    946.000000
mean      26.053188
std       23.407992
min        0.131579
25%       10.690789
50%       20.756579
75%       34.103618
max      212.500000
Name: tt_os_g_mos, dtype: float64

In [69]:
print(ib_out['pfs_m_g_status'].value_counts())
ib_out.loc[ib_out['tt_pfs_m_g_mos'] > 12, 'pfs_m_g_status'] = 0
print(ib_out['pfs_m_g_status'].value_counts())


pfs_m_g_status
1.0    489
0.0    430
Name: count, dtype: int64
pfs_m_g_status
0.0    517
1.0    402
Name: count, dtype: int64


In [70]:
print(ib_out['os_g_status'].value_counts())
ib_out.loc[ib_out['tt_os_g_mos'] > 36, 'os_g_status'] = 0
print(ib_out['os_g_status'].value_counts())


os_g_status
1    592
0    354
Name: count, dtype: int64
os_g_status
1    493
0    453
Name: count, dtype: int64


In [71]:
ib_out.to_csv('../data/crc_folfiri_out.csv', index=True)